In [9]:
import sys
sys.path.insert(1, '/Users/linusrandud/Documents/UoM/ERP/MscDissertation/Deep-Opt')

In [10]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

In [11]:
# Highly recommended to keep as cpu for problems of size <= 100
device="cpu"
print(device)
device = torch.device(device)

cpu


In [13]:
problem = QUBO("/Users/linusrandud/Documents/UoM/ERP/MscDissertation/data/qubo/bqp100.txt", 0, device)

Instance has been loaded


In [15]:
dir(problem)

['Q',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'device',
 'fitness',
 'is_valid',
 'max_fitness',
 'random_solution',
 'repair']

In [16]:
problem.Q

tensor([[  0.,   0.,   0.,  ...,  33.,   0.,   0.],
        [  0.,  52.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [ 33.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0., -43.],
        [  0.,   0.,   0.,  ...,   0., -43.,   0.]])

In [17]:
problem.fitness

<bound method QUBO.fitness of <COProblems.QUBO.QUBO object at 0x1205dd5e0>>

In [21]:
problem.max_fitness

inf

In [22]:
import torch

from COProblems.MKP import MKP
from COProblems.QUBO import QUBO
from Models.DOAE import DOAE
from OptimAE import OptimAEHandler

# Highly recommended to keep as cpu for problems of size <= 100
device = "cuda" if torch.cuda.is_available() else "cpu"
device="cpu"
print(device)
device = torch.device(device)

change_tolerance = 100
problem_size = 100
pop_size = 100
# problem = MKP("COProblems\\mkp\\problems30d.txt", "COProblems\\mkp\\fitnesses30d.txt", 12, device)
#problem = QUBO("COProblems\\qubo\\bqp500.txt", 0, device)

dropout_prob = 0.2
# 1000 bit QUBO l1 and l2:
# l1_coef = 0.0000025
# l2_coef = 0.0000025
# 500 bit QUBO l1 and l2:
# l1_coef = 0.000005
# l2_coef = 0.000005
# 100 bit l1 and l2
l1_coef = 0.0001
l2_coef = 0.0001
lr = 0.002
compression_ratio = 0.8
model = DOAE(problem_size, dropout_prob, device)
hidden_size = problem_size
handler = OptimAEHandler(model, problem, device)

population, fitnesses = handler.generate_population(pop_size)
population, fitnesses, _, _ = handler.hillclimb(population, fitnesses, change_tolerance)
handler.print_statistics(fitnesses)

cpu
Max pop fitness: 7890.0, Mean pop fitness : 7244.02978515625


In [23]:
population

tensor([[-1.,  1.,  1.,  ..., -1.,  1.,  1.],
        [-1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [-1.,  1.,  1.,  ..., -1., -1.,  1.],
        ...,
        [-1.,  1.,  1.,  ..., -1.,  1.,  1.],
        [-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])

In [24]:
fitnesses

tensor([7292., 6430., 6805., 7793., 7500., 7272., 7159., 7142., 7172., 7361.,
        7091., 6845., 7752., 6591., 7414., 6765., 7619., 7483., 7670., 7288.,
        7022., 7083., 6451., 7526., 7584., 6859., 7279., 7172., 6793., 6970.,
        6602., 7337., 7080., 7382., 7391., 7569., 7277., 7218., 7183., 7890.,
        7381., 7439., 7193., 7519., 7085., 7495., 7528., 7042., 7220., 7202.,
        7119., 7683., 7646., 7755., 6709., 7389., 7612., 6945., 7197., 7397.,
        7516., 6752., 7279., 7690., 7225., 6951., 6953., 7840., 7498., 7300.,
        7471., 6477., 6325., 7666., 5824., 7385., 7566., 7344., 7177., 7439.,
        6932., 7347., 7584., 7505., 7512., 7493., 7617., 7738., 6646., 7255.,
        7158., 6871., 7680., 7518., 7346., 6967., 7075., 7551., 7215., 7047.])

In [25]:
total_eval = 0
max_depth = 6
depth = 0

while True:
    if depth < max_depth:
        hidden_size = round(hidden_size * compression_ratio)
        model.transition(hidden_size)
        depth += 1
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_coef)
    print("Learning from population")
    # Learing with the entire population as a batch is technically not the best from a machine learning perspective,
    # but does not seem to have a massive impact on solution quality whilst also increasing learning speed significantly.
    handler.learn_from_population(population, optimizer, l1_coef=l1_coef, batch_size=pop_size)
    print("Optimising population")
    population, fitnesses, evaluations, done = handler.optimise_solutions(
        population, fitnesses, change_tolerance, encode=True, repair_solutions=True, deepest_only=False
    )
    handler.print_statistics(fitnesses)
    total_eval += evaluations
    print("Evaluations: {}".format(total_eval))
    
    # Uncomment lines below to print out best solution at every transition
    # best_i = torch.argmax(fitnesses)
    # print("Best solution - fitness = {}".format(fitnesses[best_i].item()))
    # print(population[best_i].tolist())
    if done:
        break

Learning from population
Optimising population
Max pop fitness: 7904.0, Mean pop fitness : 7623.740234375
Evaluations: 21040
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7741.4501953125
Evaluations: 45440
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7803.81005859375
Evaluations: 79093
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7846.4599609375
Evaluations: 122012
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7854.68017578125
Evaluations: 172792
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7870.47998046875
Evaluations: 233713
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7874.56005859375
Evaluations: 294433
Learning from population
Optimising population
Max pop fitness: 7970.0, Mean pop fitness : 7884.1201171875
Eval

KeyboardInterrupt: 

In [26]:
fitnesses

tensor([7970., 7970., 7970., 7970., 7970., 7888., 7970., 7970., 7970., 7970.,
        7886., 7970., 7970., 7499., 7542., 7970., 7970., 7970., 7970., 7970.,
        7970., 7970., 7790., 7970., 7970., 7970., 7970., 7970., 7540., 7970.,
        7970., 7970., 7970., 7868., 7892., 7970., 7892., 7970., 7970., 7970.,
        7964., 7970., 7892., 7970., 7970., 7970., 7970., 7938., 7762., 7970.,
        7970., 7970., 7970., 7970., 7374., 7970., 7818., 7144., 7814., 7970.,
        7970., 7970., 7970., 7970., 7970., 7970., 7970., 7970., 7970., 7970.,
        7970., 7970., 7938., 7832., 7766., 7970., 7970., 7970., 7970., 7970.,
        7970., 7970., 7970., 7970., 7970., 7970., 7970., 7970., 7970., 7970.,
        7970., 7885., 7970., 7970., 7728., 7938., 7970., 7970., 7724., 7885.])

In [27]:
population

tensor([[-1.,  1.,  1.,  ..., -1., -1.,  1.],
        [-1.,  1.,  1.,  ..., -1., -1.,  1.],
        [-1.,  1.,  1.,  ..., -1., -1.,  1.],
        ...,
        [-1.,  1.,  1.,  ..., -1., -1.,  1.],
        [-1.,  1.,  1.,  ..., -1., -1.,  1.],
        [-1.,  1.,  1.,  ..., -1., -1.,  1.]])